In [6]:
from sys import path
path.append(r'C:\Users\Izogie\Desktop\Folders\Projects\Python\KB Chat\src')
from modules.SourceManager import SourceManager
from modules.VectorDBManager import VectorDBManager

<frozen importlib._bootstrap>:488: DeprecationWarning: Type google._upb._message.MessageMapContainer uses PyType_Spec with a metaclass that has custom tp_new. This is deprecated and will no longer be allowed in Python 3.14.
<frozen importlib._bootstrap>:488: DeprecationWarning: Type google._upb._message.ScalarMapContainer uses PyType_Spec with a metaclass that has custom tp_new. This is deprecated and will no longer be allowed in Python 3.14.
c:\Python312\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [7]:
source_manager = SourceManager()
tst = source_manager.load_json("sectioned_articles.jsonl")
source_manager.data = tst
tst_docs = source_manager.to_documents()

In [40]:
from langchain_core.documents import Document
from langchain.retrievers import MultiVectorRetriever
from langchain_text_splitters import TextSplitter
from typing import Union, Optional, Sequence, Any
import uuid

class CustomParentDocRetriever(MultiVectorRetriever):
    child_splitter: Any
    """The text splitter to use to create child documents."""

    """The key to use to track the parent id. This will be stored in the
    metadata of child documents."""
    parent_splitter: Optional[TextSplitter] = None
    """The text splitter to use to create parent documents.
    If none, then the parent documents will be the raw documents passed in."""

    child_metadata_fields: Optional[Sequence[str]] = None
    """Metadata fields to leave in child documents. If None, leave all parent document 
        metadata.
    """
    def _to_document(self, data) -> Document:
        return Document(page_content=data['content'], metadata=data['metadata'])
    
    def save_txt(self, data: Union[list, set], filename: str) -> None:
        with open(filename + '.txt', 'w') as file:
            file.write(str(data))
    
    def load_txt(self, filename: str) -> list[dict]:
        with open(filename + '.txt', 'r') as file:
            return eval(file.read())
    
    def load_processed(
            self,
            load_prefix = 'semantic'):
        docs = self.load_txt(f'{load_prefix}_docs')
        full_docs = self.load_txt(f'{load_prefix}_full_docs')
        self.vectorstore.add_documents(docs)
        self.docstore.mset(full_docs)
        

    def _split_docs_for_adding(
            self,
            documents,
            save=True,
            save_prefix = 'semantic'
        ):
        # feed semantic splitter to custom ParentDocumentRetriever
        if not isinstance(documents[0], Document):
            documents = [self._to_document(doc) for doc in documents]
        full_docs = []
        docs = []
        for i, doc in enumerate(documents):
            id = str(uuid.uuid4())
            full_docs.append((id, doc))
            
            chunks = self.child_splitter.split_documents([doc])
            for child in chunks:
                if not child.page_content.startswith(f"{child.metadata['parent_article']} - "):
                    child.page_content = f"{child.metadata['parent_article']} - {child.page_content}"
                child.metadata['doc_id'] = id
                docs.append(child)
        if save:
            self.save_txt(full_docs, f'{save_prefix}_full_docs')
            self.save_txt(docs, f'{save_prefix}_docs')
        return docs, full_docs

    def add_documents(self, documents):
        docs, full_docs = self._split_docs_for_adding(documents)
        self.vectorstore.add_documents(docs)
        self.docstore.mset(full_docs)

In [20]:
from langchain_experimental.text_splitter import SemanticChunker
from langchain_huggingface import HuggingFaceEmbeddings
from langchain.storage import InMemoryStore
store = InMemoryStore()

child_splitter = SemanticChunker(HuggingFaceEmbeddings())

c:\Python312\Lib\site-packages\huggingface_hub\file_download.py:1150: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


In [12]:
from pathlib import Path
vector_manager = VectorDBManager(Path.cwd() / 'chroma_test')
vector_manager._init_langchaindb()

c:\Python312\Lib\site-packages\posthog\client.py:345: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  timestamp = datetime.utcnow().replace(tzinfo=tzutc())
c:\Python312\Lib\site-packages\posthog\client.py:345: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  timestamp = datetime.utcnow().replace(tzinfo=tzutc())


c:\Python312\Lib\site-packages\posthog\request.py:40: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  body["sentAt"] = datetime.utcnow().replace(tzinfo=tzutc()).isoformat()


In [42]:
retriever = SemanticParentDocRetriever(
    child_splitter=child_splitter,
    vectorstore=vector_manager.langdb,
    docstore=store
)

In [43]:
retriever.add_documents(tst_docs)
# retriever.load_processed()

In [44]:
retriever.invoke("What happened to Dalinar at the Rift?")

[Document(metadata={'heading': 'The Rift', 'order': 17, 'parent_article': 'Dalinar Kholin', 'keywords': ''}, page_content='Dalinar Kholin -  The Rift \nAfter some soul-searching and wishing to spare his wife pain, Dalinar approached the Rift\'s leaders in person, and Dalinar spoke with Tanalan the younger, the boy he spared who had grown up into the Brightlord of the Rift. Though initially hostile, Tanalan listened to Dalinar\'s plain-spoken assessment that the rebels would lose, and even considered Dalinar\'s offer of a personal duel between themselves. Tanalan informed Dalinar that Sadeas was working against him, and Dalinar promised him the rank of Highprince in return as the two concocted a cover story that would allow both sides to save face and call it a political victory. Dalinar left him with Tanalan\'s directions to where Sadeas\' men were hiding, pleased that he was able to find a peaceful solution, knowing that it would make Evi happy.\n\nDalinar and his men approached the c

In [46]:
from Pipeline import RAGPipeline
pipe = RAGPipeline()
# pipe.load_processed_pages()
# pipe.init_retriever()
pipe.retriever = retriever
pipe.init_compressor()

c:\Python312\Lib\site-packages\posthog\client.py:345: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  timestamp = datetime.utcnow().replace(tzinfo=tzutc())
c:\Python312\Lib\site-packages\posthog\client.py:345: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  timestamp = datetime.utcnow().replace(tzinfo=tzutc())


c:\Python312\Lib\site-packages\posthog\request.py:40: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  body["sentAt"] = datetime.utcnow().replace(tzinfo=tzutc()).isoformat()
c:\Python312\Lib\site-packages\posthog\request.py:40: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  body["sentAt"] = datetime.utcnow().replace(tzinfo=tzutc()).isoformat()
c:\Python312\Lib\site-packages\posthog\request.py:40: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  body["sentAt"] = datetime.utcnow().replace(tzinfo=tzutc()).isoformat()
c:\Python3

In [48]:
query = "What happened to Dalinar at the Rift?"
response = pipe.perform_rag(query)


Dalinar's experience at the Rift, also known as Rathalas, was a turning point in his life, marked by immense tragedy and a descent into a cycle of violence and self-destruction. 

Here's a breakdown of the events:

* **Initial Peace Efforts:** Dalinar, seeking to avoid further bloodshed, attempted to negotiate a peaceful resolution with the rebels of Rathalas, led by Tanalan the Younger. He even offered Tanalan the position of Highprince in exchange for his cooperation.
* **Tanalan's Betrayal:** Tanalan, however, lied to Dalinar, leading him into a trap that resulted in the deaths of many of Dalinar's men. Dalinar was seriously wounded in the ambush.
* **The Burning of Rathalas:** Fueled by rage and the Thrill, Dalinar unleashed a devastating attack on Rathalas, fueled by oil made by Soulcasters. He vowed to destroy the city so thoroughly that no one would ever rebuild it.
* **Evi's Death:**  In a horrifying twist, Dalinar learned that Tanalan had captured his wife, Evi, as a hostage.

In [49]:
query = "What is the difference between the fused and the singers?"
response = pipe.perform_rag(query)


The text describes a complex social hierarchy among the Fused and singers. Here's the breakdown:

**Fused:**

* **Higher status:**  Considered superior to both Regals and common singers. 
* **Nine Brands:** Divided into nine distinct groups with specialized powers.
* **Internal hierarchy:** Even within the Fused, there are levels of status and authority based on reputation, history, sanity, and the importance of their Voice (envoyform singer).
* **The Nine:** The highest authority, a council of nine Fused who hold immense power.
* **Limited rebirth:** No new Fused have been created since their initial formation.

**Singers:**

* **Common singers:**  Generally considered lower in status than Fused.
* **Regals:**  Elevated common singers granted forms of power.
* **Voices:**  Envoyform singers representing individual Fused, often accorded more respect than Regals.
* **Potential for elevation:**  While no singer has become a Fused since their inception, the possibility remains.

**Key Di

In [52]:
query = "How is Kelsier's on Roshar after dying in Mistborn 1?"
response = pipe.perform_rag(query)


The provided text focuses on Kelsier's activities in the Cognitive Realm and on Scadrial, not on Roshar. Therefore, it doesn't directly answer your question about how Kelsier is on Roshar after dying in Mistborn 1. 

However, we can infer some information:

* **Kelsier is known on Roshar as Thaidakar.** This is the name given to him by the Ghostbloods, a secret society operating across multiple planets, including Roshar. 
* **He is the leader of the Ghostbloods.** The text states that he is the "presumed founder" of the organization, which Mraize calls "the most powerful organization in all the cosmere."
* **Kelsier uses an "avatar" to visit the Ghostbloods on Roshar.** He cannot physically leave Scadrial, but he appears to be using the Ghostbloods to find a way to do so. 
* **He is interested in learning from the Heralds, particularly Kalak.**  This suggests he is trying to understand their limitations, potentially to find a way to overcome his own.

In summary, Kelsier is active on 